configuration

In [67]:
# Initial Setup:
# Import all the libraries we need

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import math
import random
import scipy as sp
from scipy import stats
import seaborn as sns


from BSE import market_session

# The next are helper functions that you will use later, if they don't make 
# much sense now, don't worry too much about it they will become clearer later:




#plot pps
def plot_pps(trial_id,time,num_traders):
    # SMA_days = 1
    pps_fname = trial_id + '_pps.csv'
    pps_b = np.empty(0)
    pps_s = np.empty(0)
    with open(pps_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        pps_b = [row[0] for row in reader]
    with open(pps_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        pps_s = [row[1] for row in reader]
        # for row in reader:
        #     pps_b = np.append(pps_b,row[0])
        #     pps_s = np.append(pps_s,row[1])
    # print(pps_s)
    ppsb = pd.Series(pps_b)
    ppss = pd.Series(pps_s)
    # ppsb = ppsb.rolling(num_traders * 60 * 60 * 24 * SMA_days).mean()
    # ppss = ppss.rolling(num_traders * 60 * 60 * 24 * SMA_days).mean()
    # ppsb = ppsb.rolling(num_traders *60 *60*9 ).mean()
    # ppss = ppss.rolling(num_traders *60 *60*9 ).mean()
    ppsb = ppsb.rolling(32400).mean()
    ppss = ppss.rolling(32400).mean()
    # ppsb = ppsb.tail(len(ppsb)-num_traders+1)
    # ppss = ppss.tail(len(ppss)-num_traders+1)
    ppsb = ppsb.dropna()
    ppss = ppss.dropna()
    ppsb = ppsb.values.flatten()
    ppss = ppss.values.flatten()
   
    # print(np.mean(ppss),np.mean(ppsb))
    pps_sum = list(np.add(ppsb,ppss))
    
    # print(len(ppsb),len(ppss))s
    # print(ppsb)
    pps_file = pd.DataFrame({'SUM': pps_sum})
    pps_file.to_csv(trial_id+'_ppsSUM.csv',index=False,header=False,sep=',')

    # fig = plt.figure(figsize=(16,9))
    # ax1 = fig.add_subplot(111, ylabel='Profit per Second')
    x = np.linspace(0 , time/60/60/24, len(ppsb))
    plt.title("Profit: PRDE")
    plt.ylabel("Profit per Second",loc='center')
    plt.xlabel("Time(days)",loc='center')
    plt.plot(x,ppsb, color='r',  label = 'B-profit' )
    plt.plot(x,ppss, color='b',  label = 'S-profit' )
    plt.plot(x,pps_sum, color='g',  label = 'SumProfit' )
    plt.legend()
    
    

# Use this to plot trades of a single experiment
def plot_trades(trial_id):
    prices_fname = trial_id + '_tape.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])/60/60
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x', color='black') 
    
# Use this to run an experiment n times and plot all trades
def n_runs_plot_trades(n, trial_id, start_time, end_time, traders_spec, order_sched):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, True, False)
        
        tdump.close()

        with open(trialId + '_tape.csv', newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                time = float(row[1])
                price = float(row[2])
                x = np.append(x,time)
                y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');

# !!! Don't use on it's own   
def getorderprice(i, sched, n, mode):
    pmin = min(sched[0][0], sched[0][1])
    pmax = max(sched[0][0], sched[0][1])
    prange = pmax - pmin
    stepsize = prange / (n - 1)
    halfstep = round(stepsize / 2.0)

    if mode == 'fixed':
        orderprice = pmin + int(i * stepsize)
    elif mode == 'jittered':
        orderprice = pmin + int(i * stepsize) + random.randint(-halfstep, halfstep)
    elif mode == 'random':
        if len(sched) > 1:
            # more than one schedule: choose one equiprobably
            s = random.randint(0, len(sched) - 1)
            pmin = min(sched[s][0], sched[s][1])
            pmax = max(sched[s][0], sched[s][1])
        orderprice = random.randint(pmin, pmax)
    return orderprice    

# !!! Don't use on it's own
def make_supply_demand_plot(bids, asks):
    # total volume up to current order
    volS = 0
    volB = 0

    fig, ax = plt.subplots()
    plt.ylabel('Price')
    plt.xlabel('Quantity')
    
    pr = 0
    for b in bids:
        if pr != 0:
            # vertical line
            ax.plot([volB,volB], [pr,b], 'r-')
        # horizontal lines
        line, = ax.plot([volB,volB+1], [b,b], 'r-')
        volB += 1
        pr = b
    if bids:
        line.set_label('Demand')
        
    pr = 0
    for s in asks:
        if pr != 0:
            # vertical line
            ax.plot([volS,volS], [pr,s], 'b-')
        # horizontal lines
        line, = ax.plot([volS,volS+1], [s,s], 'b-')
        volS += 1
        pr = s
    if asks:
        line.set_label('Supply')
        
    if bids or asks:
        plt.legend()
    plt.show()

# Use this to plot supply and demand curves from supply and demand ranges and stepmode
def plot_sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode):
    asks = []
    for s in range(seller_num):
        asks.append(getorderprice(s, sup_ranges, seller_num, stepmode))
    asks.sort()
    bids = []
    for b in range(buyer_num):
        bids.append(getorderprice(b, dem_ranges, buyer_num, stepmode))
    bids.sort()
    bids.reverse()
    
    make_supply_demand_plot(bids, asks) 

# plot sorted trades, useful is some situations - won't be used in this worksheet
def in_order_plot(trial_id):
    prices_fname = trial_id + '_tape.csv'
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            price = float(row[2])
            y = np.append(y,price)
    y = np.sort(y)
    x = list(range(len(y)))

    plt.plot(x, y, 'x', color='black')   

# plot offset function
def plot_offset_fn(offset_fn, total_time_seconds):   
    x = list(range(total_time_seconds))
    offsets = []
    for i in range(total_time_seconds):
        offsets.append(offset_fn(i))
    plt.plot(x, offsets, 'x', color='black')  

market session set up, all PRDE parameters set up 

In [68]:
# First, configure the trader specifications

trader_params_1 = {'k': 4, 's_min':-1.0, 's_max':+1.0,'F':0.8}
trader_params_2 = {'k': 7, 's_min':-1.0, 's_max':+1.0,'F':0.8}
trader_params_3 = {'k': 4, 's_min':-1.0, 's_max':+1.0,'F':0  }
trader_params_4 = {'k': 4, 's_min':-1.0, 's_max':+1.0,'F':2  }
# experimental group A or C
sellers_spec= [('PRDE', 30, trader_params_1)]
buyers_spec = sellers_spec
num_traders = 60
traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group B
# sellers_spec= [('PRDE', 30, trader_params_2)]
# buyers_spec = sellers_spec
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group D
# sellers_spec= [('PRDE', 30, trader_params_2)]
# buyers_spec = [('PRDE', 30, trader_params_1)]
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group E
# sellers_spec= [('PRDE', 30, trader_params_1)]
# buyers_spec = [('PRDE', 30, trader_params_2)]
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group F
# sellers_spec= [('PRDE', 15, trader_params_2), ('PRDE', 15, trader_params_1)]
# buyers_spec = [('PRDE', 15, trader_params_1), ('PRDE', 15, trader_params_2)]
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group G
# sellers_spec= [('PRDE', 30, trader_params_3)]
# buyers_spec = sellers_spec
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group H
# sellers_spec= [('PRDE', 30, trader_params_4)]
# buyers_spec = [('PRDE', 30, trader_params_1)]
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}


# experimental group I
# sellers_spec= [('PRDE', 30, trader_params_1)]
# buyers_spec = [('PRDE', 30, trader_params_4)]
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}

# experimental group J
# sellers_spec= [('PRDE', 15, trader_params_1), ('PRDE', 15, trader_params_4)]
# buyers_spec = [('PRDE', 15, trader_params_4), ('PRDE', 15, trader_params_1)]
# num_traders = 60
# traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}


# Next, confiure the supply and demand (and plot it)
sup_range = (60, 200)
dem_range = (1,140)

# plot_sup_dem(30, [sup_range], 30, [dem_range], 'fixed')

# Next, configure order schedules

start_time = 0
end_time = 60 * 60 * 24 * 15
supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]

order_interval = 30
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'periodic'}

# trial_id = 'test_dynamicK_day10'
# tdump = open('test_dynamicK_day10_avg_balance.csv','w')
dataA = 'test_epA'
dataB = 'test_epB'
dataC = 'test_epC'
dataD = 'test_epD'
dataE = 'test_epE'
dataF = 'test_epF'
dataG = 'test_epG'
dataH = 'test_epH'
dataI = 'test_epI'
dataJ = 'test_epJ'

tdump = open('test_epA_avg_balance.csv','w')
dump_all = True
verbose = False

# Now, run the market session
market_session(dataA, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)

tdump.close()

# Finally, plot the trades that executed during the market session
plot_trades(dataA)
plot_pps(dataA,end_time,num_traders)

Test p-value

In [65]:
data_A = pd.read_csv('test_epA_ppsSUM.csv')
 # Use seaborn to create a violin plot
data_A = data_A.dropna()
data_A = data_A.sample(n=3000,random_state=1,axis=0)
data_A = data_A.values.flatten()
statisticA, pvalueA = stats.shapiro(data_A)


data_B = pd.read_csv('test_epB_ppsSUM.csv')
data_B = data_B.dropna()
data_B = data_B.sample(n=3000,random_state=1,axis=0)
data_B = data_B.values.flatten()
statisticB, pvalueB = stats.shapiro(data_B)
if pvalueA >= 0.05:
    print('A')
if pvalueB >= 0.05:
    print('B')


print(pvalueA,pvalueB)
p_value = []
p_value.append(pvalueA)
p_value.append(pvalueB)

data_C = pd.read_csv('test_epC_ppsSUM.csv') 
data_C = data_C.dropna()
data_C = data_C.sample(n=3000,random_state=1,axis=0)
data_C = data_C.values.flatten()
statisticC, pvalueC = stats.shapiro(data_C)
if pvalueC >= 0.05:
    print('C')
print(pvalueC)
p_value.append(pvalueC)
data_D = pd.read_csv('test_epD_ppsSUM.csv')
data_D = data_D.dropna()
data_D = data_D.sample(n=3000,random_state=1,axis=0)
data_D = data_D.values.flatten()
statisticD, pvalueD = stats.shapiro(data_D)
if pvalueD >= 0.05:
    print('D')
print(pvalueD)
p_value.append(pvalueD)

data_E = pd.read_csv('test_epE_ppsSUM.csv')
data_E = data_E.dropna()
data_E = data_E.sample(n=3000,random_state=1,axis=0)
data_E = data_E.values.flatten()
statisticE, pvalueE = stats.shapiro(data_E)
if pvalueE >= 0.05:
    print('E')
print(pvalueE)
p_value.append(pvalueE)

data_F = pd.read_csv('test_epF_ppsSUM.csv')
data_F = data_F.dropna()
data_F = data_F.sample(n=3000,random_state=1,axis=0)
data_F = data_F.values.flatten()
statisticF, pvalueF = stats.shapiro(data_F)
if pvalueF >= 0.05:
    print('F')
print(pvalueF)
p_value.append(pvalueF)

data_G = pd.read_csv('test_epG_ppsSUM.csv')
data_G = data_G.dropna()
data_G = data_G.sample(n=3000,random_state=1,axis=0)
data_G = data_G.values.flatten()
statisticG, pvalueG = stats.shapiro(data_G)
if pvalueG >= 0.05:
    print('G')
print(pvalueG)
p_value.append(pvalueG)

data_H = pd.read_csv('test_epH_ppsSUM.csv')
data_H = data_H.dropna()
data_H = data_H.sample(n=3000,random_state=1,axis=0)
data_H = data_H.values.flatten()
statisticH, pvalueH = stats.shapiro(data_H)
if pvalueH >= 0.05:
    print('H')
print(pvalueH)
p_value.append(pvalueH)

data_I = pd.read_csv('test_epI_ppsSUM.csv')
data_I = data_I.dropna()
data_I = data_I.sample(n=3000,random_state=1,axis=0)
data_I = data_I.values.flatten()
statisticI, pvalueI = stats.shapiro(data_I)
if pvalueI >= 0.05:
    print('I')
print(pvalueI)
p_value.append(pvalueI)

data_J = pd.read_csv('test_epJ_ppsSUM.csv')
data_J = data_J.dropna()
data_J = data_J.sample(n=3000,random_state=1,axis=0)
data_J = data_J.values.flatten()
statisticJ, pvalueJ = stats.shapiro(data_J)
if pvalueJ >= 0.05:
    print('J')
print(pvalueJ)
p_value.append(pvalueJ)

p_value = pd.DataFrame(p_value)
p_value.to_csv('pvalue.csv',index=False,header=False,sep=',')


statistic, ab = stats.mannwhitneyu(data_B,data_A)
if ab >= 0.05:
    print('ab')
print(ab)
statistic, ac = stats.mannwhitneyu(data_C,data_A)
if ac >= 0.05:
    print('ac')
print(ac)
statistic, ad = stats.mannwhitneyu(data_D,data_A)
if ad >= 0.05:
    print('ad')
print(ad)
statistic, ae = stats.mannwhitneyu(data_E,data_A)
if ae >= 0.05:
    print('ae')
print(ae)
statistic, af = stats.mannwhitneyu(data_F,data_A)
if af >= 0.05:
    print('af')
print(af)
statistic, ag = stats.mannwhitneyu(data_G,data_A)
if ag >= 0.05:
    print('ag')
print(ag)
statistic, ah = stats.mannwhitneyu(data_H,data_A)
if ah >= 0.05:
    print('ah')
print(ah)
statistic, ai = stats.mannwhitneyu(data_I,data_A)
if ai >= 0.05:
    print('ai')
print(ai)
statistic, aj = stats.mannwhitneyu(data_J,data_A)
if aj >= 0.05:
    print('aj')
print(aj)


statistic, pvalue = stats.kruskal(data_A,data_B,data_C,data_D,data_E,data_F,data_G,data_H,data_I,data_J)
print(pvalue)


# ax = sns.violinplot(data=data_A, inner='box')
# # Label our axes appropriately
# ax.set_title("Profits by trading algorithm")
# ax.set_ylabel("Profits (GBP)")
# ax.set_xlabel("Algorithm")
# plt.show()

3.570981253363996e-34 1.145993225371763e-23
1.985130141982783e-28
7.92354211120921e-30
5.044674471569341e-44
2.2955831860117875e-31
3.1088815670862926e-34
1.6423395846412725e-12
9.75261692216143e-41
1.8103569921358546e-37
5.153225033028446e-38
5.168341819692428e-178
2.943590819783931e-287
8.598326778943866e-43
6.714617854930903e-289
1.355024260339838e-234
0.0
0.0
3.586991064896353e-66
0.0
